# Post Selection

We can do some error mitigation by throwing out the states with the wrong number of particles.  To achieve this we need a rotation in which both the total number operator $$n = \sum_i n_i \rightarrow \sum_i (I - Z_i)$$ and the Kinetic terms $$K_i = c^{\dagger}_ic_{i+1} + c^{\dagger}_{i+1}c_{i} \rightarrow X_iX_{i+1} + Y_iY_{i+1}$$ are diagonal.

In [2]:
import qiskit.quantum_info as qi
import scipy as sp
import scipy.linalg as ln
import numpy as np
import pandas as pd
import math


def Mdot(Ol):
    out = Ol[0]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])


def I(N):
    label = ['I' for i in range(N)]
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def X(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'X'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Y(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'Y'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Z(i,N):
    label = ['I' for i in range(N)]
    label[i] = 'Z'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def c(i,N):
    label_1 = ['Z' for j in range(i)]
    label_2 = ['I' for j in range(i+1,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x+1j*y)

def cd(i,N):
    label_1 = ['Z' for j in range(i)]
    label_2 = ['I' for j in range(i+1,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x-1j*y)

import numpy as np

def Mdot(Ol):
    out = Ol[0]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])

This rotation will be composed of $$\sqrt{i\text{SWAP}}_{ij} = e^{i \frac{\pi}{2}(X_iX_j + Y_iY_j)} $$ and $$T_i = e^{i\frac{\pi}{8}Z_i}$$

The complete rotation is $$ U_{ij} = \sqrt{i\text{SWAP}}_{ij}T_iT^{\dagger}_j $$

In [56]:
riswap = 1/2*(I(2) + Mdot([Z(0,2),Z(1,2)]) + np.cos(np.pi/4)*(I(2) - Mdot([Z(0,2),Z(1,2)])) + 1j*np.sin(np.pi/4)*(Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)])))

riswap_tst = ln.expm(1j*np.pi/8*( Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)]) ))

np.amax(np.abs( riswap - riswap_tst ))

1.1102230246251565e-16

In [49]:
T0 = np.cos(np.pi/8)*I(2) + 1j*np.sin(np.pi/8)*Z(0,2)
T1 = np.cos(np.pi/8)*I(2) - 1j*np.sin(np.pi/8)*Z(1,2)

Um = Mdot([riswap,T0,T1])
Umd = np.conjugate(np.transpose(Um))

pd.DataFrame( Mdot([Um, 1/2*(Mdot([X(0,2),X(1,2)]) + Mdot([Y(0,2),Y(1,2)]) ) ,Umd]) )

,0,1,2,3
0,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j
1,0.0+0.0j,1.000000e+00+0.000000e+00j,-5.551115e-17-1.665335e-16j,0.0+0.0j
2,0.0+0.0j,-5.551115e-17+1.665335e-16j,-1.000000e+00+0.000000e+00j,0.0+0.0j
3,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j


In [54]:
pd.DataFrame( Mdot([Um, Z(0,2) + Z(1,2) ,Umd]) )

,0,1,2,3
0,2.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,0.0+0.0j,0.0+0.0j,0.0+0.0j,-2.0+0.0j


In order to perform the $\sqrt{i\text{SWAP}}_{ij}$ gate using only two qubit gates we either have to expand $\sqrt{i\text{SWAP}}_{ij}$ in terms of many gates or we can re-run the slatter determinate circuit with new qubit indcices so that $i$ and $j$ are neighbors in the quantum hardware.  